In [194]:
import numpy as np
from numpy import asarray
from PIL import Image

In [5]:
def compareCard_deprecated(subArray, COLOR):
    sum = 0
    for column in range(subArray.shape()[0]):
        for row in range(subArray.shape()[1]):
            if subArray[column,row] == COLOR:
                sum +=1
    result = sum/ ( subArray.shape()[0]*subArray.shape()[1] )
    return result

In [6]:
def detection_deprecated(image, background_color, backside_color):

    BACKGROUND_COLOR = [253,1,255]
    BACKSIDE_COLOR = backside_color

    card_shape = (32,32)
    
    card_list = []

    for column in range(image.shape()[0] - card_shape[0]):
        for row in range(image.shape()[1] - card_shape[1]):
            subArray = image[column:column+card_shape[0], row:row+card_shape[1]]
            if compareCard(subArray,BACKGROUND_COLOR) == 0:
                card_list.append(subArray)
    
    for card in card_list:
        if compareCard(card,BACKSIDE_COLOR) == 0:
            return card

In [7]:
image = Image.open('board.jpg')
print(image.format)
print(image.size)
print(image.mode)

arrayImage = asarray(image)

BACKGROUND_COLOR = [253,1,255]

JPEG
(508, 508)
RGB


In [21]:
arrayImage[8,3]

array([245,   7, 240], dtype=uint8)

In [66]:
image = Image.open('MemoryGame\\model\\background.png')

arrayImage = asarray(image)

rgb_weights = [0.2989, 0.5870, 0.1140]   #Pourquoi ces valeurs ? et pourquoi pas ? (stackoverflow), pas le temps d'analyser pourquoi
grayscale_card = np.dot(arrayImage[...,:3], rgb_weights)

print(grayscale_card)

from PIL import Image
import numpy as np

w, h = 512, 512
img = Image.fromarray(np.uint8(mat * 255), 'L')
img = Image.fromarray(grayscale_card)
print(type(img))
print(img)
"""img.convert("rgb")
img.save('my.jpg')
img.show()"""

[[249.8735 249.8735 247.5776 ... 250.9919 251.6685 250.2665]
 [247.0695 251.9765 252.893  ... 254.4045 249.5655 251.7825]
 [252.306  244.8768 230.7023 ... 245.7734 251.1059 253.1057]
 ...
 [252.8822 231.0245 203.8434 ... 228.3435 252.893  247.8935]
 [252.2215 244.0159 231.7085 ... 244.0249 252.0105 250.6085]
 [250.1185 250.8195 251.1104 ... 253.024  249.9075 251.3095]]
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=F size=336x336 at 0x1B8E4FEA280>


'img.convert("rgb")\nimg.save(\'my.jpg\')\nimg.show()'

In [181]:
def filter_image(image):
    #on commence par convertir l'image en gris
    #on utilise le produit d'une matrice par un vecteur pour transformer le vecteur dim(3) des 3 RGB
    #en vecteur dim(1)
    
    #Pourquoi ces valeurs ? et pourquoi pas ? (stackoverflow), pas le temps d'analyser pourquoi  
    """image = Image.fromarray(image.astype(np.uint8))
    grayscale_image = np.array(image.convert('L'))
    print(img)
    grayscale_image = asarray(img)"""
    #rgb_weights = np.asarray([0.2989, 0.5870, 0.1140])
    #grayscale_image = np.dot(image[...,:3], rgb_weights)

    #on applique un threshold pour avoir uniquement 0 ou 255
    formatted_card = np.copy(image)
    for x,column in enumerate(image):
        for y,pixel in enumerate(column):
            formatted_card[x,y] = 0 if np.dot(pixel,[0.2989, 0.5870, 0.1140]) < 128 else 255
    print(formatted_card.shape)
    return formatted_card

In [182]:
def detection(image, background_image):
    BORDER_COLOR = 0
    BORDER_SIZE = 3
    
    card_shape = (156, 156)
    
    card_list = []
    
    filtered_background = filter_image(image)
    """filtered_background_image = Image.fromarray(background_image)
    filtered_background_image.show()"""
    
    face_up_card = []

    #on parcours le board
    for column in range(0,image.shape[0] - card_shape[0]):
        for row in range(0,image.shape[1] - card_shape[1]):
            
            #on extrait une subarray de la taille d'une carte
            subArray = image[column:column+card_shape[0],row:row+card_shape[1]]
            
            #On extrait les bordures de subarray
            borders = np.asarray([subArray[index_x][index_y] 
                                  for index_x in range(card_shape[0])
                                  for index_y in range(card_shape[1])
                       if min(index_x,index_y) < BORDER_SIZE or 
                          min(card_shape[0] - index_x,card_shape[1] - index_y) < BORDER_SIZE])
            
            #On fait la moyenne de couleur sur ce border, si c'est OK alors c'est une carte
            avg = np.average(borders)
            
            if avg >= 240: #arbitrary but this is the threshold, cannot assume a dark pixel is 255 because of compression
                #On enlève le border de subarray
                card = subArray[BORDER_SIZE:card_shape[0]-BORDER_SIZE,
                                BORDER_SIZE:card_shape[1]-BORDER_SIZE]
                card_list.append(card)

                #On compare subarray et l'arrière des cartes
                filtered_card = filter_image(card)
                #print(filtered_card)
                #print(filtered_background)
                average_distance = np.average(np.absolute(filtered_card - filtered_background))
                
                #si c'est pas un arrière de carte on met dans un tableau
                if average_distance < 10:
                    face_up_card.append(card)
                
                
    #Renvoyer le tableau avec toutes les cartes face up
    return card_list

In [183]:
image = Image.open('board.jpg')
background  = Image.open('MemoryGame\\model\\background.png')

arrayImage = asarray(image)
arrayBackground = asarray(background)

card_list = detection(arrayImage, arrayBackground)

(508, 508, 3)


KeyboardInterrupt: 

In [180]:
card_list

[]

In [285]:
def naiveCardSegmentation(board,):
    cards_positions = np.array([[14,14],[14,180],[14,346],
                                [180,14],[180,180],[180,346],
                                [346,14],[346,180],[346,346]])
    card_shape = (150,150)
    
    cards = []
    
    board = np.array(board)
    
    for card in range(len(cards_positions)):
        cards.append(board[cards_positions[card,0]:cards_positions[card,0]+card_shape[0]-1,
                           cards_positions[card,1]:cards_positions[card,1]+card_shape[1]-1])
    return cards

In [198]:
image = Image.open('board.jpg')
background  = Image.open('MemoryGame\\model\\background.png')

arrayImage = asarray(image)
arrayBackground = asarray(background)

card_list = naiveCardSegmentation(arrayImage, arrayBackground)

In [255]:
def applyThreshold(image1,threshold):
    formatted_card = np.copy(image1)
    for x,column in enumerate(image1):
        for y,pixel in enumerate(column):
            formatted_card[x,y] = 0 if pixel < threshold else 255
    return formatted_card

In [287]:
def imageComparison(image1, image2):
    
    img1 = Image.fromarray(np.array(image1))
    grayscale_image1 = np.array(img1.convert('L'))
    #grayscale_image1 = applyThreshold(grayscale_image1,128)
    
    img2 = Image.fromarray(np.array(image2))
    grayscale_image2 = np.array(img2.convert('L'))
    #grayscale_image2 = applyThreshold(grayscale_image2,128)
    
    average_distance = np.mean(np.square(np.absolute(grayscale_image1 - grayscale_image2)))
    
    return average_distance < 10

In [283]:
def naiveDetection(board):
    background = Image.open("background.png")
    arrayBackground = asarray(background)
    
    cards = naiveCardSegmentation(board)
    
    return [card for card in cards if not imageComparison(background,card)]

In [276]:
img = Image.fromarray(card_list[8])
#img.show()
background  = Image.open('MemoryGame\\model\\background.png')
arrayBackground = asarray(background)

imageComparison(card_list[2],card_list[7])

for card in card_list:
    print(imageComparison(card_list[2],card))

False
False
True
True
False
True
True
True
False


In [270]:
card_list[2]
image = Image.fromarray(card_list[2].astype(np.uint8))
image.save("background.png")

In [288]:
image = Image.open('board.jpg')
faceup = naiveDetection(image)

In [290]:
for image in faceup:
    img = Image.fromarray(image)
    img.show()